# Computation Graph

This page gives an overview of the general representation of the computation graph.

It must support multiple features:

- static typing
- variables (read / write / pointers)
- 0 to many number of return values per operation
- function calls

Some other features will be added later, but design should already take them into account:
- dynamic typing
- control flow (loops, conditions)

## Value

A Value represent an actual value represented in the graph.
It can be:
- a placeholder / argument
- a constant
- the output of an operation.

There is no variable type. A placeholder / const can be of type `ptr`, which can be read / written, and acts like a variable.

In [1]:
class Value:
    
    def __init__(self, name, ty):
        # value name
        self.name = name
        
        # graph the node belong to
        self.graph = graph
        
        # value type (might be tensor, pointer, void, or others)
        self.ty = ty
        
        # Operation defining self (might be none)
        self.opdef = None
        
        # Operations that have self as input
        self.users = []
        
class ConstValue(Value):
    
    def __init__(self, val):
        # val transformed to understandable representation for the framework
        # stored as data in the host memory
        # will be copied to device memory only at exec time
        pass
    
    # returns true if it's tensor with all values same
    def is_splat(self): pass

class PlaceholderValue(Value):
    
    def __init__(self, ty):
        # Value will be given at execution time
        # Must be of same type as ty
        pass
    
class OpValue(Value):
    
    def __init__(self, op, output_idx):
        # op is the operation creating this value
        # output_idx is the index of self in the list of outputs of op
        pass

# Types

Every value has a type, can be any of:

- Void: Not sure it's really useful yet, an op that returns nothing just returns no value, and not a value type void, but maybe could be useful for some special ops / templating.
- StaticTensor: Tensor with known datatype and known shape
- DynamicTensor: Tensor with known datatype and unknown shape.

Dynamic tensors are needed for some special operations (eg tile with non-constant value).  
Maybe could also be used for parameters with unknown types.

## Operation

An operation takes any number of values as inputs, and produce any nomber of values as outputs.  
It also takes static arguments, knownn when building the ops. It can be things such as padding sizes for conv, or axes for permute or sum.
They are operations for all types of operations:  
Eg: add, mul, sub

### Versions

An operation may have several versions.  
The common use case is to have one version for every type (eg 1 version for add f32, 1 version for add f64).  
Usually the only difference between versions is the actual implementation. (all versions have it's own implementation).  
The version is chosen automatically when the op is built, based on the inputs and attrs.

In [4]:

class Op:
    
    def __init__(self, inputs, outputs, attrs, name):
        # operation name
        self.name = name
        
        # the graph the op belongs to
        self.graph = graph
        
        # Operation version
        self.version = self.select_version()
        
        #input nodes
        self.inputs = inputs
        
        #output nodes for the operation (can be empty)
        self.outputs = outputs
        
        # static-time operation attributes
        self.attrs = attrs

## Graph

The graph represents a whole group of operations and nodes.  
It may optionally keep tracks or not or all the ops / values created.  
Usually the main / global graph is used to build ndoes by the user, and don't need to keep track of them (this way the nodes can be deleted as long as they are not referenced anymore).  
Other graphs, used for optimization purposes, usually only contains nodes always usefull, and can be tracked of on the class.

Despite storing nodes, graph has lots of usesful methods to:

- build consts / placehodlers values
- build operations
- build operations to compute the gradient
- execute nodes

In [1]:
class Graph:
    
    def __init__(self):
        # If true, keeps the references to all objects in the graph, otherwhise it keeps no data
        self.hold_refs = True
        self.vals = dict()
        self.ops = dict()
        pass
    
    def build_placeholder(self, ty, name):
        pass
    
    def build_const(self, val, name):
        pass
    
    def build_op_add(self, lhs, rhs, name):
        pass
    
    def build_grad_ops(self, loss_val, vals):
        # vals is a list of value
        # return a new list of value grads
        # grads[i] = dloss_val/dvals[i]
        pass
        
    def run(self, feeds, target_vals, target_ops):
        # Compute all values in the list target_vals
        # Execute all operations in the list target_ops
        # feeds is dict<Value, nparray> contains the placeholders values need to run the ops
        # returns list<nparray> with the computed values for target_vals
        pass

## Graph serialization

Nodes of the graph can be serialized to text form (JSON).  
This text form can be unserialized back to add nodes to a graph.  
Every op / value is given an id, to avoid rebuilding nodes several times when serializing / unserializing again.

## Graph execution

Here is what happens when the graph run metod is called.

## Select and clone a subgraph

Select and clone the subgraph of all ops that need to be executed. It uses some form of value caching, where it's not needed to recompute some values that were computed before and didn't change since last computation.  
A value didn't change usually when none of it's predecessors was updated.  
For example, one of the targets might have been evaluated already. So no need to recompute it.  
Also, one intermediate value might have been a target before, so there is no need to recompute it. This OpValue is then replaced by a PlaceholderValue.  

At first, can do a simple implementation that just clones the whole required subgraph, no caching.

### Renaming

Some ops and values need to be renamed when cloning to make sure all nodes have unique names.

## Graph optimizations / compilations

This pass is really backend dependant, Can do optimizations such as:
- lowering
- fusing
- change to other representations.

Then it can also pre-compiles the graph to be ready for execution.  
It returns black box objects, that just takes inputs, execute the graph, and return outputs.

## Graph execution

The compiled graph is then used to get the results.  
One optimization might be to cache this compiled graph, and reuse it directly, but might be complicated.  
Maybe a feature for layer.

## Graph execution through modular implementations

Every implementation contains both the methods to build the graph, and it's own compiler / executor.  

This technique can be used to build the graph in one implementation / env / language, and then use another implementation to run it.  
For example the Python API could target any implementation made on another languages.  
Implementations can be added dynamically, without the need to recompile anything.  

This is done using a middle-end C API, that allows to connect 2 implementations together. 
One implementation can be connected to make the graph run on other implementations.  
And one implementation can be connected to allow other implementations to run this graph using this one. (can be both).  

To make it possible, there must be a way:
- to convert values from current implementation to the native general representation
- to convert values from the native general representation to the current implementation.
- the front implementation must call the right native functions to use other implementations
- the back implementation must call the right naive functions to allow being used by other implementations.  

The graph is serialized / unseralized to be passed between the different implementations.  
There could be a optimization that doesn't serialize / unserialize nodes already seralized before, later feature.  

This is done right when calling the run method. This way no optimization happened yet, the graph is going to be rebuild on the back implementation, and it's going to go through all the execution steps mentionned above, before returning the results to the original implementation.


## Caching

These are some ideas for caching internal nodes and avoid recomputing.  
It can only be done by the backend, I'll think more about it later.

Every value is cached, to avoid creating nodes that compute the same value multiple times.
Every value returns an hash, if a value with the same hash exist, this one is returned instead of build a new one.

- Placeholders with same types / tensor shape match to the same hash, unless 2 different placeholders try to use the same one.
- If constant is a splat, or of size < 10, it has a hash based on this. Otherwhise, all consts have a different hash.
- Op output has an hash depending on the inputs nodes and op propperties. If inputs and properties has same, hash value is the same.

### Placeholders

Actually for placeholders it's a little more complicated. Sometimes, there could be more than one hit for the same node. (There could be 2 or more placeholders cached with same tensor shape/type).  
Chosing one over the other could lead to creating lots of node, but maybe the other one had them all cached already.  
The solution is to start matching by generate all possibles assignments, and try them them, only couting the number of cache hits. The one with the biggest number is selected, and mathcing proceeds with this.  

### Consts

Maybe a special thing could be done for consts too. To be able to cache all const values, no matter the shape, and use a better way to find one through the cache. Maybe store using a hash, and if there is a hit, check if it's the same tensor.  
Or check using numpy array address ?

### General hashing

One solution would be to give a unique id to every value.  
Placeholders and Consts would have their id found using the technique described earlier.
Every op will have it's own cache, and the key is a list of integers, being hashed. These number are the inputs ids, and the attributes, converted to integers.